In [1]:
import openai

# replace YOUR_API_KEY with your actual API key for the ChatGPT service
openai.api_key = ""

import os 
os.environ["OPENAI_API_KEY"] = ""

import json


https://medium.com/mlearning-ai/using-chatgpt-for-question-answering-on-your-own-data-afa33d82fbd0

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):

    messages = [
        {
        "role": "user", 
        "content": prompt}
        ]

    completion = openai.ChatCompletion.create(

    model=model,

    messages=messages,

    temperature=0,# hatGPT responses are indeterministic. It means every time you call the API you will get slightly different response. You can make it deterministic by passing the parameter temperate to 0

    )

    return completion.choices[0].message["content"]

# answer question from provided document

In [3]:
# %pip install langchain duckdb unstructured chromadb openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.document_loaders.unstructured import UnstructuredFileLoader 

loader = UnstructuredFileLoader('outfit_choices.txt')

documents = loader.load()

In [20]:
print(documents)

[Document(page_content='outfit 1 is designed for Beach party, Summer barbecue, Vacation, Day at the beach, Casual summer gathering, Pool party, Music festival, Weekend outing, Picnic, Cruise. It contains {Top: A crisp white button-up shirt with a navy blue v-neck sweater layered over it. Bottom: Khaki chinos or trousers. Shoes: Brown leather loafers. Accessories: A striped or solid-colored silk tie, a leather belt, and a classic wristwatch.}\n\noutfit 2 is designed for Casual gathering, Concert, Sport event, Weekend outing, Casual shopping, City exploration, Skate park, Music festival, Road trip, Streetwear event. It contains { Top: Bomber jacket or a varsity jacket. Layering: Plain or striped t-shirt. Bottom: Baggy jeans or cargo pants. Footwear: High-top sneakers or boots. Accessories: Baseball cap worn backward, a chain necklace, and a beanie. }\n\noutift 30 is designed for Gym, Casual Friday at work, Casual shopping, Sport event, Weekend outing, Road trip, Casual gathering, Walking

In [3]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [4]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

In [6]:
from langchain.chains import VectorDBQA
from langchain.chat_models import ChatOpenAI

qa = VectorDBQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", vectorstore=db, k=1)

c:\Users\fangy\miniconda3\envs\bert_nlp\lib\site-packages\langchain\chains\retrieval_qa\base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [11]:
query = "What to wear for a party?"
qa.run(query)

"It depends on the type of party you're attending. If it's a beach party, summer barbecue, vacation, day at the beach, casual summer gathering, pool party, music festival, weekend outing, picnic, or cruise, you can consider outfit 1. It consists of a crisp white button-up shirt with a navy blue v-neck sweater layered over it, khaki chinos or trousers, brown leather loafers, a striped or solid-colored silk tie, a leather belt, and a classic wristwatch. \n\nIf it's a casual gathering, concert, sport event, weekend outing, casual shopping, city exploration, skate park, music festival, road trip, or streetwear event, you can consider outfit 2. It includes a bomber jacket or a varsity jacket, a plain or striped t-shirt for layering, baggy jeans or cargo pants, high-top sneakers or boots, a baseball cap worn backward, a chain necklace, and a beanie. \n\nIf you provide more details about the party, I can help you choose the appropriate outfit."

In [8]:
query1 = "What to wear for a party? Format it in python dictionary"
response1 = qa.run(query1)
print(response1)

{
   "party":{
      "outfit 1":{
         "Top":"A crisp white button-up shirt with a navy blue v-neck sweater layered over it.",
         "Bottom":"Khaki chinos or trousers.",
         "Shoes":"Brown leather loafers.",
         "Accessories":[
            "A striped or solid-colored silk tie",
            "a leather belt",
            "a classic wristwatch"
         ]
      },
      "outfit 2":{
         "Top":"Bomber jacket or a varsity jacket.",
         "Layering":"Plain or striped t-shirt.",
         "Bottom":"Baggy jeans or cargo pants.",
         "Footwear":"High-top sneakers or boots.",
         "Accessories":[
            "Baseball cap worn backward",
            "a chain necklace",
            "a beanie"
         ]
      }
   }
}


In [10]:
y = json.loads(response1)
y['party']['outfit 1']

{'Top': 'A crisp white button-up shirt with a navy blue v-neck sweater layered over it.',
 'Bottom': 'Khaki chinos or trousers.',
 'Shoes': 'Brown leather loafers.',
 'Accessories': ['A striped or solid-colored silk tie',
  'a leather belt',
  'a classic wristwatch']}

In [13]:
query9 = "What to wear for a party? Tell me the outfit number only based on the provided context"
response9 = qa.run(query9)
print(response9)

Based on the provided context, outfit 1 is suitable for a party.


In [13]:
query2 = "What to wear for a gym? Format it in python dictionary"
response2 = qa.run(query2)
print(response2)

{
  "outfit_number": 30,
  "occasion": "Gym",
  "items": [
    "Sports Luxe: Blending sportswear and luxury elements",
    "joggers",
    "hoodies",
    "sneakers"
  ]
}


In [16]:
query3 = "What to wear for a wedding? Describe the outfit in terms of cateogry, color and texture. Format it in python dictionary"
response3 = qa.run(query3)
print(response3)

Category: Formal attire
Color: Classic black or dark navy
Texture: Smooth and refined

Outfit:
{
  "Top": "Tailored black or navy suit jacket",
  "Bottom": "Matching suit trousers",
  "Shirt": "Crisp white dress shirt",
  "Tie": "Silk tie in a complementary color",
  "Shoes": "Polished black leather shoes",
  "Accessories": "Matching belt, cufflinks, and a sleek wristwatch"
}


In [17]:
query3 = "What to wear for a wedding? Format it in python dictionary without newspace"
response3 = qa.run(query3)
print(response3)

{
  "Top": "Crisp white button-up shirt",
  "Layering": "Navy blue v-neck sweater",
  "Bottom": "Khaki chinos or trousers",
  "Shoes": "Brown leather loafers",
  "Accessories": "Striped or solid-colored silk tie, leather belt, classic wristwatch"
}


In [18]:
query2 = "What to wear for a gym? Format it in python dictionary without newspace"
response2 = qa.run(query2)
print(response2)

{"outfit 30": "Sports Luxe: Blending sportswear and luxury elements, sports luxe style combines athletic-inspired clothing like joggers, hoodies, and sneakers with high-end fabrics, tailored cuts, and refined details."}


In [19]:
prompt4 = "What to wear for a gym? Describe the outfit in terms of 'Outerwear', 'Top', 'Bottom', 'Footwear', 'Accessories'. Format response in python dictionary without newspace" 

response4 = qa.run(prompt4)
print(response4)

{
  "Outerwear": "A lightweight zip-up hoodie or a moisture-wicking jacket.",
  "Top": "A breathable and moisture-wicking athletic t-shirt or tank top.",
  "Bottom": "Comfortable and flexible workout shorts or leggings.",
  "Footwear": "Proper athletic shoes with good support and grip.",
  "Accessories": "A sweatband or headband to keep sweat away from the face, and a water bottle to stay hydrated."
}


## key-take-away

Need to provide a text or other format resources to describe outfit. Then use prompt to let GPT answer questions based on the provided resources. 

Several important things: 
1) The description of outfit need to be high-quality to allow chatgpt to provide detailed outfit suggestion. 
2) chatgpt is simply doing semantic search among the occasions of outfit then repeat the description of outfit. \
-To save GPT quota, we may not need to provide outfit description but only occasion description to GPT. \
-Once GPT decides the outfit number, we can simply call out the detailed outfit description from our database. 
3) If chatGPT is simply doing semantic search, the use of chatgpt to generate outfit suggestion might not be necessary. \
-First, we can ask chatgpt to generate query like "occasion - season - style" then search in our outfit dataset, instead of let chatgpt suggest outfit directly. \
-Net, use prompt template to directly output our prepared outfit images and let user choose one\
-Finally go back to search for product based on the selected outfit description. 

Note: we need to be careful about how GPT accounts for token usage


# Ways to directly output outfit from database

https://towardsdatascience.com/use-langchains-output-parser-with-chatgpt-for-structured-outputs-cf536f692685

In [14]:
from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template(
    "I suggest the following outfit {content}."
)


In [18]:
outfit = '''\nTop: A crisp white button-up shirt with a navy blue v-neck sweater layered over it.
Bottom: Khaki chinos or trousers.
Shoes: Brown leather loafers.
Accessories: A striped or solid-colored silk tie, a leather belt, and a classic wristwatch.'''

response = prompt_template.format(content= outfit)

In [19]:
print(response)

I suggest the following outfit 
Top: A crisp white button-up shirt with a navy blue v-neck sweater layered over it.
Bottom: Khaki chinos or trousers.
Shoes: Brown leather loafers.
Accessories: A striped or solid-colored silk tie, a leather belt, and a classic wristwatch..
